**Hidden Markov models for cracking codes**

In this exercise you have to make a partially built HMM work and use it to solve some simple substitution ciphers. Plaintext data is provided in 'plaintext' directory. Encrypted data is in 'encrypted'. Some of the texts were originally English some of them were Russian; the sequences are also of different lengths. 

This homework is worth **15 points** and is due by the next class (**24th Oct.**), please submit the results of the **TASK 5** (a list of files and names of the author/work) to Anytask in the following format: 'filename author' where 'filename' is a file from "encrypted/\*_encrypted.txt" and 'author' is a file from "plaintext/\*.txt" (not including 'english.txt', 'russian.txt' or 'all.txt') which best matches the decrypted text.




In [44]:
# Utilities for loading data from file and converting characters to integers and back.
import numpy as np
    
def get_char_to_int_mapping(path):
    # Load data from path and get mapping from characters to integers and back.
    characters = set()
    for line in open(path):
        characters.update(set([c for c in line.strip()]))
    char_to_int_mapping = dict([(char, i) for i, char in enumerate(sorted(list(characters)))])
    int_to_char_mapping = [char for char, i in char_to_int_mapping.items()]
    return char_to_int_mapping, int_to_char_mapping

def load_sequences(path, char_to_int_mapping):
    # Load data from path and map to integers using mapping.
    return [[char_to_int_mapping[c] for c in line.strip()] for line in open(path)]

def estimate_markov_model_from_sequences(sequences, num_states):
    # Estimate a Markov model based on the sequences (integers) provided.

    # pi[i] = Pr(s_0 = i)
    pi_counts = np.zeros(num_states)

    # A[i, j] = Pr(s_t = j | s_{t-1} = i)
    A_counts = np.zeros((num_states, num_states))
    move_out_count = np.zeros(num_states)

    for n, sequence in enumerate(sequences):
        for prev, nxt in zip(sequence[:-1], sequence[1:]):
            A_counts[prev, nxt] += 1
            pi_counts[prev] += 1
        if len(sequence) > 0:
            pi_counts[sequence[-1]] += 1
            
    pi = pi_counts / np.sum(pi_counts)
    A = A_counts / np.sum(A_counts, axis=1)[:, None]
    
    return pi, A

**TASK 1**: Make the following block run by completing the method 'estimate_markov_model_from_sequences' above.

In [45]:
%%time
# Some data to use.
plaintext = 'plaintext/english.txt'
# plaintext = 'plaintext/shakespeare.txt'
# plaintext = 'plaintext/russian.txt'

ciphertext = 'encrypted/1_encrypted.txt' # short sequences in english
# ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian

# load a character to integer mapping and reverse                                                                                                         
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)

# load sequences as ints                                                                                                                                  
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)
print(plaintext_sequences[:2])
# estimate a markov model over characters                                                                                                                 
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))


[[9], [20, 8, 5, 0, 16, 5, 18, 9, 15, 4]]
CPU times: user 5.51 s, sys: 40 ms, total: 5.55 s
Wall time: 5.54 s


In [46]:
%%time
pi.shape, A.shape, pi.sum(), A.sum(axis=1)


CPU times: user 45 µs, sys: 0 ns, total: 45 µs
Wall time: 47.7 µs


((27,),
 (27, 27),
 1.0,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

Below is a mostly implemented HMM.

In [47]:
from IPython.display import clear_output

In [98]:
class HMM():

    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping) # number of states = number of chars 
        self.num_outputs = len(observations_to_char_mapping) # number of outputs ?
        self.states_to_char_mapping = states_to_char_mapping # get char from state
        self.observations_to_char_mapping = observations_to_char_mapping # what is observations ??? 
       
        # Random initialization
        self.pi = np.random.rand(self.num_states)# init pi with random 
        self.pi /= np.sum(self.pi)
        self.A = np.random.rand(self.num_states, self.num_states).astype('float128') # init transition matrix with random 
        self.A /= np.sum(self.A, 1, keepdims=True)
        self.B = np.random.rand(self.num_states, self.num_outputs).astype('float128') # init emission matrix 
        self.B /= np.sum(self.B, 1, keepdims=True)
        
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        self.pi = parameters['pi'] if self.fixed_pi else self.pi 
        
        self.fixed_A = 'A' in parameters
        self.A = parameters['A'] if self.fixed_A else self.A
        
        self.fixed_B = 'B' in parameters
        self.B = parameters['B'] if self.fixed_B else self.B
           
        previous_llh = None # prev log-likelihood
        
        iteration = 0
        while True and iteration < max_iters: # why we dont use for-loop here? 
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences) # E-step from EM-algo

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts) # M-step from EM-algo

            # Output some sequences for debugging.
            if iteration % 5 == 0:
                self.output(sequences[:10])

            # Log likelihood should be increasing
            print('iteration %d; log likelihood %.4f' % (iteration, log_likelihood))
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
            previous_llh = log_likelihood
        
            iteration += 1
        self.output(sequences[:10])

    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            # Run Forward-Backward dynamic program
            alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sequence)
  
            # Accumulate statistics.
            pi_counts += gamma[:, 0]
            A_counts += xi
            for t, x in enumerate(sequence):
                B_counts[:, x] += gamma[:, t]
            
            total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = self.forward(sequence) 
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        ss = np.sum(alpha * beta, 0, dtype=np.float128)
        if (ss == 0).any():
            print('err')
        assert np.isfinite(ss).all()
        gamma = (alpha * beta) / ss

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = np.zeros_like(self.A)
            this_xi += (self.A * alpha[:, t][:, None]) * (beta[:, t+1] * self.B[:, sequence[t+1]])

#             for i in range(self.num_states):
#                 this_xi[i, :] += alpha[i, t] * self.A[i, :] * beta[:, t+1] * self.B[:, sequence[t+1]]
#                 for j in range(self.num_states):
#                     this_xi[i, j] += alpha[i, t] * self.A[i, j] * beta[j, t+1] * self.B[j, sequence[t+1]]        
            xi += this_xi / np.sum(this_xi)
                
        return alpha, beta, gamma, xi, np.log(np.sum(alpha[:, len(sequence)-1]))

    def forward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)), dtype=np.float128)
#         print(alpha.shape)
        alpha[:, 0] = self.pi * self.B[:, sequence[0]]
        for t in range(len(sequence) - 1):
            x = sequence[t+1]
            
#             alpha[:, t+1] = self.B[:, x] * np.sum(self.A * alpha[:, t][:, None], axis=0) 
            alpha[:, t+1] = self.B[:, x] * np.sum(self.A * alpha[:, t][:, None], axis=0, dtype=np.float128)
#             for i in range(self.num_states):
# #                 sum_val = sum([alpha[j, t] * self.A[j, i] for j in range(self.num_states)])
#                 sum_val = np.sum(self.A[:, i] * alpha[:, t], dtype=np.float128)
# #                 print(alpha.dtype, A.dtype)
# #                 assert sum_val == floatsum_sec), "{} {}".format(sum_val, sum_sec)
#                 alpha[i, t+1] = self.B[i, x] * sum_val

        return alpha 
    
    def backward(self, sequence):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = np.zeros((len(self.pi), len(sequence)))
        beta[:, -1] = 1
        
#         for i in range(self.num_states):
#             beta[i, -1] = 1
        
        for i, x in enumerate(sequence[1:][::-1]):
            t = len(sequence) - 1 - i
#             beta[:, t-1] = np.sum(self.A * (beta[:, t] * self.B[:, x])[:, None], axis=1) # check 

            beta[:, t-1] = np.sum(self.A * (beta[:, t] * self.B[:, x]), axis=1, dtype=np.float128)
            
#             for i in range(self.num_states):
#                 beta[i, t-1] = sum([beta[j, t] * self.A[i, j] * self.B[j, x] for j in range(self.num_states)]) 

        return beta

    def output(self, sequences):
        # Output some decoded states. 
        clear_output()
        for i, sequence in enumerate(sequences):
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))


In [99]:
mt = np.ones((2,3))
np.sum(mt, axis=1, keepdims=True)

array([[3.],
       [3.]])

**TASK 2**: Implement the assertions in 'forward' and 'backward' methods on the HMM class so that the following block passes.

In [100]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping

# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)

# Estimate the parameters and decode the encrypted sequences.
hmm.estimate_with_em(encrypted_sequences[:100], parameters={'pi': pi, 'A': A})

(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous inge me no more i thall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp


In [88]:
np.float128

numpy.float128

**TASK 3**: Some of the encrypted sequences are quite long. Try decoding some from 'encrypted/99_encrypted.txt' (note these are in Russian).

**TASK 4**: Make your implementation of forward and backward more efficient by removing all but the outermost for-loop.

**TASK 5**: Try to classify the author of each text. 